<a href="https://colab.research.google.com/github/CalculatedContent/ww-colab/blob/master/ww_kdd2020_200_300.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Setup**

In [0]:
import pandas as pd
import numpy as np
import time
import datetime as dt
import itertools
import os
import statsmodels.api as sm
import matplotlib.pyplot as plt
from google.colab import files #Google colab ONLY

In [0]:
istart = 200
iend = istart+100

In [0]:
## If export the output to Google Drive (colab ONLY) --> Optional
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import sys
path = '/content/drive/My Drive/kdd2020_cv'
sys.path.append(path)

In [0]:
!ls "$path"

wwdata-0.2.3


In [0]:
!pip install weightwatcher #install from source
#!pip install powerlaw #needed if install ww from local

  Created wheel for powerlaw: filename=powerlaw-1.4.6-cp36-none-any.whl size=24786 sha256=d42fcbaf84f7696de1e98ffa21e7337aa7866103471b6d364b827147032f2331
  Stored in directory: /root/.cache/pip/wheels/e0/27/02/08d0e2865072bfd8d7c655e290521e3feca3fc22f1ac460601
Successfully built powerlaw


In [0]:
import weightwatcher as ww

# Suppress the powerlaw package warnings
# "powerlaw.py:700: RuntimeWarning: divide by zero encountered in true_divide"
# "powerlaw.py:700: RuntimeWarning: invalid value encountered in true_divide"
import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)

# crea
import logging
logger = logging.getLogger("imported_module")
logger.setLevel(logging.CRITICAL)

Using TensorFlow backend.


## **PyTorch Models**

### **1. Analyze all models in Pytorchcv**

In [0]:
!pip install pytorchcv

     |████████████████████████████████| 399kB 3.0MB/s 


In [0]:
## Check storage
#import subprocess
#p = subprocess.Popen('df -h', shell=True, stdout=subprocess.PIPE)
#print(str(p.communicate()[0], 'utf-8'))

In [0]:
from pytorchcv.model_provider import get_model as ptcv_get_model
import torch

import pytorchcv
import pytorchcv.models.model_store

modelnames = pytorchcv.models.model_store._model_sha1.keys()
modelnames = list(modelnames)
print('{} models to examine'.format(len(modelnames)))

483 models to examine


In [0]:
# this_models = pd.read_excel('notfound.xlsx')[0].tolist()
# len(this_models[1:])

In [0]:
## Takes about 10-15 hours to run 100 models as of 9/3/2019

data = {}
ss = time.time()
i = 0
notfound = []
ww_error = []
for modelname in modelnames[istart:iend]: 
    print(modelname)
    try:
        s = time.time()
        model = ptcv_get_model(modelname, pretrained=True)

    except:
        notfound.append(modelname)
        e = time.time()
        print("{} not found".format(modelname))
        print('--------------------------------------------------------')
        i += 1
        continue

    try:
        watcher = ww.WeightWatcher(model=model, logger=logger)
        results = watcher.analyze(alphas = True, spectralnorms=True, softranks=True, mp_fit = True)
        e = time.time()
        
        data[modelname]=watcher.get_summary()
        
        print('time spent is {}'.format((e-s)/60.))
        print('--------------------------------------------------------')
        
    except:
        ww_error.append(modelname)
        e = time.time()
        print("Error in weightwatcher operation for {}".format(modelname))
        print('--------------------------------------------------------')
        
    i += 1

    #Export/backup the results every 1 hour runtime or every 50 models  
    if ((e - ss) > 60*60) or ((i - istart)%25 == 0): 
      
        ddf = pd.DataFrame.from_dict(data).transpose()
        ddf.to_excel(path + '/data_{}-{}_{}.xlsx'.format(istart, i, dt.date.today()))
        
        ss = time.time()
        
        #try:
        #    !rm -rf /root/.torch/models #clean the memory
        #except:
        #    pass
          
    if i >= iend: #Export when all models are finished
      
        ddf = pd.DataFrame.from_dict(data).transpose()
        ddf.to_excel(path + '/data_{}-{}_{}.xlsx'.format(istart, i, dt.date.today()))
        
        # break

ddf = pd.DataFrame.from_dict(data).transpose()
ddf.to_excel(path + '/data_{}-{}_{}.xlsx'.format(istart, i, dt.date.today()))

mobilenetv2_w3d4
time spent is 0.3092291990915934
--------------------------------------------------------
mobilenetv2_w1
time spent is 0.3751315633455912
--------------------------------------------------------
mobilenetv3_large_w1
time spent is 0.5644406119982401
--------------------------------------------------------
igcv3_wd4
time spent is 0.2678507685661316
--------------------------------------------------------
igcv3_wd2
time spent is 0.3006654461224874
--------------------------------------------------------
igcv3_w3d4
time spent is 0.37885146141052245
--------------------------------------------------------
igcv3_w1
time spent is 0.4594446380933126
--------------------------------------------------------
mnasnet_b1
time spent is 0.4016276359558105
--------------------------------------------------------
mnasnet_a1
time spent is 0.4392171223958333
--------------------------------------------------------
darts
time spent is 1.285421605904897
------------------------------------